### UT Bot Strategy Overview

The **UT Bot** is a trend-following strategy that generates buy and sell signals based on the **ATR (Average True Range)** and **Exponential Moving Average (EMA)**. Here's a breakdown of how it works:

1. **ATR Calculation**:
   - The strategy begins by calculating the **ATR** over a defined period (e.g., 10 days). The ATR is used to measure volatility in the market, providing the foundation for setting dynamic stop-loss levels.
   
2. **nLoss**:
   - The `nLoss` variable is a factor of **ATR**, scaled by a sensitivity factor (e.g., 1). This determines the amount by which the stop-loss should trail the price.

3. **ATRTrailingStop**:
   - The **ATRTrailingStop** is a dynamic stop level that moves with the price. It adjusts based on the ATR and the market's volatility:
     - If the price is trending upwards and the previous close is above the stop, the stop is moved up.
     - If the price is trending downwards and the previous close is below the stop, the stop is moved down.
     - This ensures that the stop follows the price movement while allowing for some volatility.
   
4. **Buy Signal**:
   - A **buy** signal is generated when the price closes above the **ATRTrailingStop** level and the **EMA** crosses above the ATRTrailingStop. This suggests a potential upward trend, prompting a buy order.

5. **Sell Signal**:
   - A **sell** signal is generated when the price closes below the **ATRTrailingStop** level and the **EMA** crosses below the ATRTrailingStop. This suggests a potential downward trend, prompting a sell order.

By combining **ATR** for volatility-based stop levels and **EMA** for trend direction, the UT Bot aims to capture significant price movements while minimizing risk through trailing stops.


### Implement Strategy Overview

#### Core Logic
- **Entry Signals**:
  - `Buy Signal`: Enters long position when price crosses above ATR trailing stop
  - `Sell Signal`: Enters short position when price crosses below ATR trailing stop
  
- **Position Management**:
  - Uses `ReverseReduce` handling for opposite signals:
    - Automatically closes existing position when receiving opposite signal
    - Immediately opens reverse position in new direction
  - No simultaneous long/short positions
  - Daily frequency rebalancing

#### Key Characteristics
1. **Trend Following**  
   Based on volatility-adjusted trailing stop (ATR) for trend identification
   
2. **Always In Market**  
   Maintains either long or short position at all times

3. **Automatic Reversal**  
   Flips position direction on opposing signals:

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta

ticker = 'BTC-USD'  
start_date = datetime.now() - timedelta(days=365) 
end_date = end=datetime.now()   

df = yf.download(ticker, start=start_date, end=end_date, interval='1d')

if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.droplevel(1)

df.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2024-02-23,50731.949219,51497.933594,50561.777344,51283.906250,21427078270
2024-02-24,51571.101562,51684.195312,50585.445312,50736.371094,15174077879
2024-02-25,51733.238281,51950.027344,51306.171875,51565.214844,15413239245
2024-02-26,54522.402344,54938.175781,50931.031250,51730.539062,34074411896
2024-02-27,57085.371094,57537.839844,54484.199219,54519.363281,49756832031


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Download Bitcoin data
ticker = 'BTC-USD'
start_date = datetime.now() - timedelta(days=400)
end_date = datetime.now()

df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.droplevel(1)
df.columns = df.columns.str.lower()  # Ensure lowercase column names

def calculate_heikin_ashi(df):
    ha_df = df.copy()
    
    # Heikin Ashi Close
    ha_df['ha_close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    
    # Heikin Ashi Open
    ha_df['ha_open'] = 0.0
    for i in range(len(ha_df)):
        if i == 0:
            ha_df.iat[0, ha_df.columns.get_loc('ha_open')] = df['open'].iloc[0]
        else:
            ha_df.iat[i, ha_df.columns.get_loc('ha_open')] = (ha_df['ha_open'].iloc[i-1] + ha_df['ha_close'].iloc[i-1]) / 2
    
    # Heikin Ashi High/Low
    ha_df['ha_high'] = ha_df[['high', 'ha_open', 'ha_close']].max(axis=1)
    ha_df['ha_low'] = ha_df[['low', 'ha_open', 'ha_close']].min(axis=1)
    
    return ha_df

use_heikin_ashi = False  # Set to True for Heikin Ashi signals
if use_heikin_ashi:
    df = calculate_heikin_ashi(df)
    src = df['ha_close']
else:
    src = df['close']

def calculate_rma(series, length):
    alpha = 1 / length
    rma = np.zeros_like(series)
    
    # Pine-compatible initialization
    rma[length-1] = series[0:length].mean()  # Bars 0-9 for length=10
    
    for i in range(length, len(series)):
        rma[i] = alpha * series[i] + (1 - alpha) * rma[i-1]
    return rma

# Calculate True Range
df['tr'] = np.max([
    df['high'] - df['low'],
    abs(df['high'] - df['close'].shift(1)),
    abs(df['low'] - df['close'].shift(1))
], axis=0)

# Calculate 10-period ATR using RMA
atr_length = 10
df['atr'] = calculate_rma(df['tr'], atr_length)

a = 1
df['nLoss'] = a * df['atr']

# Initialize Trailing Stop
df['xATRTrailingStop'] = np.nan

for i in range(1, len(df)):
    prev_stop = df['xATRTrailingStop'].iloc[i-1] if not np.isnan(df['xATRTrailingStop'].iloc[i-1]) else 0
    current_src = src.iloc[i]
    prev_src = src.iloc[i-1]
    nLoss = df['nLoss'].iloc[i]

    if current_src > prev_stop and prev_src > prev_stop:
        new_stop = max(prev_stop, current_src - nLoss)
    elif current_src < prev_stop and prev_src < prev_stop:
        new_stop = min(prev_stop, current_src + nLoss)
    else:
        if current_src > prev_stop:
            new_stop = current_src - nLoss
        else:
            new_stop = current_src + nLoss
    
    df['xATRTrailingStop'].iat[i] = new_stop

# Calculate 1-period EMA of src
df['ema1'] = src.ewm(span=1, adjust=False).mean()

# Calculate crossovers using EMA1 and xATRTrailingStop
df['above'] = (df['ema1'] > df['xATRTrailingStop']) & (df['ema1'].shift(1) <= df['xATRTrailingStop'].shift(1))
df['below'] = (df['ema1'] < df['xATRTrailingStop']) & (df['ema1'].shift(1) >= df['xATRTrailingStop'].shift(1))

# Buy/Sell signals require src to be above/below the trailing stop AND EMA crossover
df['buy_signal'] = (src > df['xATRTrailingStop']) & df['above']
df['sell_signal'] = (src < df['xATRTrailingStop']) & df['below']

# Create figure
fig = go.Figure()

# Candlestick plot
candle_colors = np.where(src > df['xATRTrailingStop'], 'green', 'red')
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    increasing_line_color='green',
    decreasing_line_color='red',
    showlegend=False
))

# Trailing Stop Line
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['xATRTrailingStop'],
    mode='lines',
    line=dict(color='blue', width=2),
    name='Trailing Stop'
))

# Buy Signals
buy_signals = df[df['buy_signal']]
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['low'] * 0.98,
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy'
))

# Sell Signals
sell_signals = df[df['sell_signal']]
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['high'] * 1.02,
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell'
))

# Layout settings
fig.update_layout(
    title='Bitcoin Price with UT Bot Alerts',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

[*********************100%***********************]  1 of 1 completed
C:\Users\OMEN\AppData\Local\Temp\ipykernel_53884\1530854279.py:52: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



### Import Libraries

In [ ]:
# First install vectorbt if needed
# pip install vectorbt

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib
import plotly.graph_objects as go
from datetime import datetime, timedelta
import vectorbt as vbt
import datetime as dt
import json  # Import the json module
import requests  # Also make sure to import the requests module
from plotly.subplots import make_subplots

### Yahoo Finance - Ut Bot Alerts

In [ ]:
# Download Bitcoin data
ticker = 'BTC-USD'
start_date = datetime.now() - timedelta(days=90)
end_date = datetime.now()

df = yf.download(ticker, start=start_date, end=end_date, interval='1d')
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.droplevel(1)
df.columns = df.columns.str.lower()  # Ensure lowercase column names

# Parameters
SENSITIVITY = 1
ATR_PERIOD = 10

# Calculate ATR and nLoss
df['xATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=ATR_PERIOD)
df['nLoss'] = SENSITIVITY * df['xATR']

# Drop rows with NaN values in nLoss/xATR
df = df.dropna(subset=['nLoss'])

# Function to compute ATRTrailingStop
def xATRTrailingStop_func(close, prev_close, prev_atr, nloss):
    if close > prev_atr and prev_close > prev_atr:
        return max(prev_atr, close - nloss)
    elif close < prev_atr and prev_close < prev_atr:
        return min(prev_atr, close + nloss)
    elif close > prev_atr:
        return close - nloss
    else:
        return close + nloss
    
# Initialize and calculate ATRTrailingStop
df['xATRTrailingStop'] = np.nan
df.iloc[0, df.columns.get_loc('xATRTrailingStop')] = 0.0  # Set initial value

for i in range(1, len(df)):
    current_close = df['close'].iloc[i]
    prev_close = df['close'].iloc[i-1]
    prev_atr = df['xATRTrailingStop'].iloc[i-1]
    current_nLoss = df['nLoss'].iloc[i]
    
    new_stop = xATRTrailingStop_func(current_close, prev_close, prev_atr, current_nLoss)
    df.iat[i, df.columns.get_loc('xATRTrailingStop')] = new_stop

# Calculating signals
ema = vbt.MA.run(df["close"], 1, short_name='EMA', ewm=True)
 
df["above"] = ema.ma_crossed_above(df["xATRTrailingStop"])
df["below"] = ema.ma_crossed_below(df["xATRTrailingStop"])
 
df["buy"] = (df["close"] > df["xATRTrailingStop"]) & (df["above"]==True)
df["sell"] = (df["close"] < df["xATRTrailingStop"]) & (df["below"]==True)

# Create plot
fig = go.Figure()

# Candlestick trace
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='Price'
))

# Trailing Stop line
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['xATRTrailingStop'],
    mode='lines',
    name='Trailing Stop',
    line=dict(color='blue', width=2)
))

# Buy signals
buy_signals = df[df['buy']]
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['low'] * 0.98,
    mode='markers',
    name='Buy',
    marker=dict(
        symbol='triangle-up',
        size=10,
        color='green',
        line=dict(width=1, color='DarkSlateGrey')
    )
))

# Sell signals
sell_signals = df[df['sell']]
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['high'] * 1.02,
    mode='markers',
    name='Sell',
    marker=dict(
        symbol='triangle-down',
        size=10,
        color='red',
        line=dict(width=1, color='DarkSlateGrey')
    )
))

# Update layout
fig.update_layout(
    title='UT Bot Alerts - Bitcoin',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template='plotly_dark'
)

fig.show()

[*********************100%***********************]  1 of 1 completed


In [16]:
# Create portfolio from signals
# pf = vbt.Portfolio.from_signals(
#     df["close"],
#     entries=df["buy"],          # Long entries
#     short_entries=df["sell"],   # Short entries
#     upon_opposite_entry='ReverseReduce',  # Close opposite position before opening new
#     freq='d',                   # Daily frequency
# )

# Run the strategy
pf = vbt.Portfolio.from_signals(
    df["close"],              
    entries=df["buy"],       
    exits=df["sell"],    
    freq="h"                    
)
# Get and display detailed trade information
trades_df = pf.trades.records_readable  # Get trades in readable format
print("\nDetailed Trade List:")
print(trades_df.to_string())  # Convert to string for better formatting

# Show performance stats
print("\nPerformance Statistics:")
print(pf.stats())


Detailed Trade List:
     Exit Trade Id  Column      Size           Entry Timestamp  Avg Entry Price  Entry Fees            Exit Timestamp  Avg Exit Price  Exit Fees       PnL    Return Direction  Status  Position Id
0                0       0  0.001029 2024-11-30 05:00:00+00:00     97163.593750         0.0 2024-11-30 06:00:00+00:00    96668.414062        0.0 -0.509635 -0.005096      Long  Closed            0
1                1       0  0.001029 2024-11-30 15:00:00+00:00     96726.078125         0.0 2024-11-30 22:00:00+00:00    96491.296875        0.0 -0.241491 -0.002427      Long  Closed            1
2                2       0  0.001030 2024-12-01 03:00:00+00:00     96366.039062         0.0 2024-12-01 23:00:00+00:00    97287.140625        0.0  0.948657  0.009558      Long  Closed            2
3                3       0  0.001022 2024-12-02 01:00:00+00:00     98062.906250         0.0 2024-12-02 03:00:00+00:00    96634.703125        0.0 -1.459292 -0.014564      Long  Closed            

### Backtesting Results for BTCUSDT using Yahoo Finance(3/6/9 months)

**Interval 1d**

| **Metrics**                        | **Value (3 months)**  | **Value (6 months)**  | **Value (9 months)**  |
|------------------------------------|-----------------------|-----------------------|-----------------------|
| **Start Date**                     | 2024-12-09            | 2024-09-10            | 2024-06-12            |
| **End Date**                       | 2025-02-25            | 2025-02-25            | 2025-02-25            |
| **Total Return (%)**               | -1.08%                | 28.39%                | 19.85%                |
| **Max Gross Exposure (%)**         | 100.00%               | 100.00%               | 100.00%               |
| **Max Drawdown (%)**               | 5.17%                 | 11.16%                | 17.20%                |
| **Total Trades**                   | 3                     | 10                    | 16                    |
| **Total Closed Trades**            | 3                     | 10                    | 16                    |
| **Total Open Trades**              | 0                     | 0                     | 0                     |
| **Best Trade (%)**                 | 4.27%                 | 23.09%                | 23.09%                |
| **Worst Trade (%)**                | -5.17%                | -5.17%                | -6.73%                |

**Interval 1h**

| **Metrics**                        | **Value (3 months)**  | **Value (6 months)**  | **Value (9 months)**  |
|------------------------------------|-----------------------|-----------------------|-----------------------|
| **Start Date**                     | 2024-11-29            | 2024-08-31            | 2024-06-02            |
| **End Date**                       | 2025-02-25            | 2025-02-25            | 2025-02-25            |
| **Total Return (%)**               | -11.33%               | 8.27%                 | 7.98%                 |
| **Max Gross Exposure (%)**         | 100.00%               | 100.00%               | 100.00%               |
| **Max Drawdown (%)**               | 17.59%                | 17.59%                | 17.59%                |
| **Total Trades**                   | 151                   | 302                   | 440                   |
| **Total Closed Trades**            | 150                   | 301                   | 439                   |
| **Total Open Trades**              | 1                     | 1                     | 1                     |
| **Best Trade (%)**                 | 4.96%                 | 7.44%                 | 7.44%                 |
| **Worst Trade (%)**                | -3.56%                | -3.56%                | -3.56%                |

### Binance - UT Bot Alerts

In [9]:
URL = 'https://api.binance.com/api/v3/klines'
 
intervals_to_secs = {
    '1m':60,
    '3m':180,
    '5m':300,
    '15m':900,
    '30m':1800,
    '1h':3600,
    '2h':7200,
    '4h':14400,
    '6h':21600,
    '8h':28800,
    '12h':43200,
    '1d':86400,
    '3d':259200,
    '1w':604800,
    '1M':2592000
}
 
def download_kline_data(start: dt.datetime, end:dt.datetime ,ticker:str, interval:str)-> pd.DataFrame:
    start = int(start.timestamp()*1000)
    end = int(end.timestamp()*1000)
    full_data = pd.DataFrame()
     
    while start < end:
        par = {'symbol': ticker, 'interval': interval, 'startTime': str(start), 'endTime': str(end), 'limit':1000}
        data = pd.DataFrame(json.loads(requests.get(URL, params= par).text))
 
        data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.iloc[:,0]]
        data=data.astype(float)
        full_data = pd.concat([full_data,data])
         
        start+=intervals_to_secs[interval]*1000*1000
         
    full_data.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume','Close_time', 'Qav', 'Num_trades','Taker_base_vol', 'Taker_quote_vol', 'Ignore']
     
    return full_data

# UT Bot Parameters
SENSITIVITY = 1
ATR_PERIOD = 10
 
# Ticker and timeframe
TICKER = "BTCUSDT"
INTERVAL = "1h"
 
# Backtest start/end date
START = datetime(2024, 11, 29)
END   = datetime(2025, 2, 26)

# Get data from Binance
pd_data = download_kline_data(START, END, TICKER, INTERVAL)

# Compute ATR And nLoss variable
pd_data["xATR"] = talib.ATR(pd_data["High"], pd_data["Low"], pd_data["Close"], timeperiod=ATR_PERIOD)
pd_data["nLoss"] = SENSITIVITY * pd_data["xATR"]
 
#Drop all rows that have nan, X first depending on the ATR preiod for the moving average
pd_data = pd_data.dropna()
pd_data = pd_data.reset_index()

# Function to compute ATRTrailingStop
def xATRTrailingStop_func(close, prev_close, prev_atr, nloss):
    if close > prev_atr and prev_close > prev_atr:
        return max(prev_atr, close - nloss)
    elif close < prev_atr and prev_close < prev_atr:
        return min(prev_atr, close + nloss)
    elif close > prev_atr:
        return close - nloss
    else:
        return close + nloss
 
# Filling ATRTrailingStop Variable
pd_data["ATRTrailingStop"] = [0.0] + [np.nan for i in range(len(pd_data) - 1)]
 
for i in range(1, len(pd_data)):
    pd_data.loc[i, "ATRTrailingStop"] = xATRTrailingStop_func(
        pd_data.loc[i, "Close"],
        pd_data.loc[i - 1, "Close"],
        pd_data.loc[i - 1, "ATRTrailingStop"],
        pd_data.loc[i, "nLoss"],
    )

# Calculating signals
ema = vbt.MA.run(pd_data["Close"], 1, short_name='EMA', ewm=True)
 
pd_data["Above"] = ema.ma_crossed_above(pd_data["ATRTrailingStop"])
pd_data["Below"] = ema.ma_crossed_below(pd_data["ATRTrailingStop"])
 
pd_data["Buy"] = (pd_data["Close"] > pd_data["ATRTrailingStop"]) & (pd_data["Above"]==True)
pd_data["Sell"] = (pd_data["Close"] < pd_data["ATRTrailingStop"]) & (pd_data["Below"]==True)

# Run the strategy
pf = vbt.Portfolio.from_signals(
    pd_data["Close"],              
    entries=pd_data["Buy"],       
    exits=pd_data["Sell"],    
    freq="h"                    
)


# Get and display detailed trade information
trades_df = pf.trades.records_readable  # Get trades in readable format
print("\nDetailed Trade List:")
print(trades_df.to_string())  # Convert to string for better formatting

pf.stats()


Detailed Trade List:
     Exit Trade Id  Column      Size  Entry Timestamp  Avg Entry Price  Entry Fees  Exit Timestamp  Avg Exit Price  Exit Fees       PnL    Return Direction  Status  Position Id
0                0       0  0.001038                7         96355.39         0.0              15        96968.29        0.0  0.636083  0.006361      Long  Closed            0
1                1       0  0.001036               27         97102.32         0.0              32        96137.48        0.0 -0.999953 -0.009936      Long  Closed            1
2                2       0  0.001031               37         96645.41         0.0              44        96466.11        0.0 -0.184848 -0.001855      Long  Closed            2
3                3       0  0.001032               49         96336.75         0.0              69        97185.18        0.0  0.875859  0.008807      Long  Closed            3
4                4       0  0.001024               71         97964.93         0.0           

Start                                                 0
End                                                2126
Period                                 88 days 15:00:00
Start Value                                       100.0
End Value                                     98.944866
Total Return [%]                              -1.055134
Benchmark Return [%]                          -9.592704
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              14.652203
Max Drawdown Duration                  47 days 13:00:00
Total Trades                                        124
Total Closed Trades                                 124
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  39.516129
Best Trade [%]                                 4.951181
Worst Trade [%]                               -2

In [5]:
# Print the actual start date
print("Backtest start date:", START.strftime('%Y-%m-%d %H:%M:%S'))
print("Backtest end date:", END.strftime('%Y-%m-%d %H:%M:%S'))

Backtest start date: 2024-11-29 00:00:00
Backtest end date: 2025-02-26 00:00:00


In [4]:
# Create figure
fig = go.Figure()

# Add price and indicator traces
fig.add_trace(go.Candlestick(
    x=pd_data['index'],
    open=pd_data['Open'],
    high=pd_data['High'],
    low=pd_data['Low'],
    close=pd_data['Close'],
    name='Price',
    increasing_line=dict(color='green'),
    decreasing_line=dict(color='red'),
))

fig.add_trace(go.Scatter(
    x=pd_data['index'],
    y=pd_data['ATRTrailingStop'],
    name='ATR Trailing Stop',
    line=dict(color='blue', width=1.5)
))

# Add EMA trace (note: 1-period EMA = Close price - you might want to adjust period)
fig.add_trace(go.Scatter(
    x=pd_data['index'],
    y=ema.ma,
    name='EMA',
    line=dict(color='purple', width=1.5, dash='dot')
))

# Add buy/sell signals
buy_signals = pd_data[pd_data['Buy']]
sell_signals = pd_data[pd_data['Sell']]

fig.add_trace(go.Scatter(
    x=buy_signals['index'],
    y=buy_signals['Close'],
    mode='markers',
    name='Buy',
    marker=dict(
        symbol='triangle-up',
        color='limegreen',
        size=10,
        line=dict(width=1, color='DarkSlateGrey')
    )
))

fig.add_trace(go.Scatter(
    x=sell_signals['index'],
    y=sell_signals['Close'],
    mode='markers',
    name='Sell',
    marker=dict(
        symbol='triangle-down',
        color='tomato',
        size=10,
        line=dict(width=1, color='DarkSlateGrey')
    )
))

# Format layout
fig.update_layout(
    title=f'{TICKER} Trading Signals',
    xaxis_title='Date',
    yaxis_title='Price (USDT)',
    xaxis=dict(
        type='date',
        tickformat='%Y-%m-%d',
        rangeslider=dict(visible=False)
    ),
    hovermode='x unified',
    template='plotly_dark',
    height=600,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Add ATR subplot
fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=pd_data['index'],
    y=pd_data['xATR'],
    name='ATR',
    line=dict(color='cyan', width=1)
))

fig2.update_layout(
    title='ATR Indicator',
    xaxis=dict(tickformat='%Y-%m-%d'),
    height=300,
    template='plotly_dark'
)

combined_fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                           vertical_spacing=0.05,
                           row_heights=[0.7, 0.3])

# Add main chart traces
for trace in fig.data:
    combined_fig.add_trace(trace, row=1, col=1)

# Add ATR trace
combined_fig.add_trace(go.Scatter(
    x=pd_data['index'],
    y=pd_data['xATR'],
    name='ATR',
    line=dict(color='cyan', width=1)
), row=2, col=1)

# Update layout
combined_fig.update_layout(
    title_text=f'{TICKER} Strategy Analysis',
    height=600,
    template='plotly_dark',
    xaxis=dict(rangeslider=dict(visible=False)),
    showlegend=True
)

combined_fig.update_yaxes(title_text="Price", row=1, col=1)
combined_fig.update_yaxes(title_text="ATR", row=2, col=1)

combined_fig.show()

### Backtesting Results for BTCUSDT using Binance(3/6/9 months)

**Interval 1d**

| **Metrics**                        | **Value (3 months)**  | **Value (6 months)**  | **Value (9 months)**  |
|------------------------------------|-----------------------|-----------------------|-----------------------|
| **Start Date**                     | 2024-11-29            | 2024-08-31            | 2024-06-02            |
| **End Date**                       | 2025-02-26            | 2025-02-26            | 2025-02-26            |
| **Total Return (%)**               | -1.19%                | 30.05%                | 31.89%                |
| **Max Gross Exposure (%)**         | 100.00%               | 100.00%               | 100.00%               |
| **Max Drawdown (%)**               | 5.19%                 | 10.97%                | 17.28%                |
| **Total Trades**                   | 3                     | 8                     | 14                    |
| **Total Closed Trades**            | 3                     | 8                     | 14                    |
| **Total Open Trades**              | 0                     | 0                     | 0                     |
| **Best Trade (%)**                 | 4.22%                 | 23.07%                | 23.07%                |
| **Worst Trade (%)**                | -5.16%                | -5.16%                | -6.72%                |

**Interval 1h**

| **Metrics**                        | **Value (3 months)**  | **Value (6 months)**  | **Value (9 months)**  |
|------------------------------------|-----------------------|-----------------------|-----------------------|
| **Start Date**                     | 2024-11-29            | 2024-08-31            | 2024-06-02            |
| **End Date**                       | 2025-02-26            | 2025-02-26            | 2025-02-26            |
| **Total Return (%)**               | -1.06%                | 10.57%                | 14.74%                |
| **Max Gross Exposure (%)**         | 100.00%               | 100.00%               | 100.00%               |
| **Max Drawdown (%)**               | 14.65%                | 14.65%                | 14.65%                |
| **Total Trades**                   | 124                   | 255                   | 366                   |
| **Total Closed Trades**            | 124                   | 255                   | 366                   |
| **Total Open Trades**              | 0                     | 0                     | 0                     |
| **Best Trade (%)**                 | 4.95%                 | 7.27%                 | 7.64%                 |
| **Worst Trade (%)**                | -2.50%                | -2.50%                | -2.80%                |


### Alpaca Signal Checking

In [6]:
# First install required packages:
# pip install alpaca-py talib vectorbt plotly numpy pandas

from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta, timezone
import pandas as pd
import talib
import numpy as np
import vectorbt as vbt
import plotly.graph_objs as go

# **************************
# ALPACA API CONFIGURATION
# **************************
API_KEY = "PK8JLD7B6WKBQ0DXXZAA"
SECRET_KEY = "FOyWhJ3IvLLfv9Z0PbiIpNJt7QQ0amZpwa2OcuhR"
SYMBOL = "BTC/USD"

# Initialize Alpaca client
client = CryptoHistoricalDataClient(API_KEY, SECRET_KEY)

# **************************
# DATA FETCHING
# **************************
# Set time range (Alpaca requires UTC timezone-aware datetimes)
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=100)

# Create and send request
request_params = CryptoBarsRequest(
    symbol_or_symbols=[SYMBOL],
    timeframe=TimeFrame.Day,
    start=start_date,
    end=end_date
)

# Get data and convert to DataFrame
bars = client.get_crypto_bars(request_params)
df = bars.df

# **************************
# DATA PREPROCESSING
# **************************
# Clean up DataFrame structure
if isinstance(df.index, pd.MultiIndex):
    df = df.droplevel(0)  # Remove symbol from index if present
df = df.sort_index(ascending=True)  # Ensure chronological order
df.columns = df.columns.str.lower()  # Standardize column names

# **************************
# INDICATOR CALCULATION
# **************************
# Parameters
SENSITIVITY = 1
ATR_PERIOD = 10

# Calculate ATR and normalized loss
df['xATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=ATR_PERIOD)
df['nLoss'] = SENSITIVITY * df['xATR']
df = df.dropna(subset=['nLoss'])  # Remove initial NaN values

# **************************
# TRAILING STOP CALCULATION
# **************************
def xATRTrailingStop_func(close, prev_close, prev_atr, nloss):
    if close > prev_atr and prev_close > prev_atr:
        return max(prev_atr, close - nloss)
    elif close < prev_atr and prev_close < prev_atr:
        return min(prev_atr, close + nloss)
    elif close > prev_atr:
        return close - nloss
    else:
        return close + nloss

# Initialize trailing stop column
df['xATRTrailingStop'] = np.nan
df.iloc[0, df.columns.get_loc('xATRTrailingStop')] = df['close'].iloc[0]  # Set initial value

# Calculate trailing stops iteratively
for i in range(1, len(df)):
    current_close = df['close'].iloc[i]
    prev_close = df['close'].iloc[i-1]
    prev_atr = df['xATRTrailingStop'].iloc[i-1]
    current_nLoss = df['nLoss'].iloc[i]
    
    new_stop = xATRTrailingStop_func(current_close, prev_close, prev_atr, current_nLoss)
    df.iat[i, df.columns.get_loc('xATRTrailingStop')] = new_stop

# **************************
# SIGNAL GENERATION
# **************************
# Calculate EMA and signals
ema = vbt.MA.run(df["close"], 1, short_name='EMA', ewm=True)
df["above"] = ema.ma_crossed_above(df["xATRTrailingStop"])
df["below"] = ema.ma_crossed_below(df["xATRTrailingStop"])
df["buy"] = (df["close"] > df["xATRTrailingStop"]) & df["above"]
df["sell"] = (df["close"] < df["xATRTrailingStop"]) & df["below"]

# **************************
# VISUALIZATION
# **************************
# Create plot figure
fig = go.Figure()

# Price Candles
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name='Price',
    increasing_line_color='#2ECC71',
    decreasing_line_color='#E74C3C'
))

# Trailing Stop Line
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['xATRTrailingStop'],
    mode='lines',
    name='ATR Trailing Stop',
    line=dict(color='#3498DB', width=2)
))

# Create signal filters first
buy_signals = df[df['buy']]
sell_signals = df[df['sell']]

# Buy Signals (corrected with comma)
fig.add_trace(go.Scatter(
    x=buy_signals.index,
    y=buy_signals['low'] * 0.98,
    mode='markers',
    name='Buy',
    marker=dict(
        symbol='triangle-up',
        size=10,
        color='#2ECC71',  # Added comma here
        line=dict(width=1, color='DarkSlateGrey')
    )
))

# Sell Signals (corrected with comma)
fig.add_trace(go.Scatter(
    x=sell_signals.index,
    y=sell_signals['high'] * 1.02,
    mode='markers',
    name='Sell',
    marker=dict(
        symbol='triangle-down',
        size=10,
        color='#E74C3C',  # Added comma here
        line=dict(width=1, color='DarkSlateGrey')
    )
))

# Layout Configuration
fig.update_layout(
    title=f'ATR Trailing Stop Strategy - {SYMBOL}',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template='plotly_dark',
    hovermode='x unified',
    height=800,
    margin=dict(l=50, r=50, b=50, t=50)
)

# Show plot
fig.show()